# Db2 Go API Documentation
The following document summarizes the Go language support for Db2.

## Database APIs

1.	[Open a Connection to Database](#OpenApi)
2.	[Prepare a Statement](#PrepareApi)
3.	[SQL Query](#QueryApi)
4.	[Execute a Prepared Statement](#ExecApi)
5.	[Begin a Transaction](#BeginApi)
6.	[Close a Connection](#CloseApi)
7.	[Commit a Transaction](#CommitApi)
8.	[Rollback a Transaction](#RollbackApi)
9.	[Query a Single Row](#QueryRowApi)
10.	[Return Answer Set Column Names](#ColumnsApi)
11.	[Read the Next Row](#NextApi)
12.	[Copy Row Values into Answer Set](#ScanApi)

## API Conventions
All of the Db2 GO functions either return an error code, or a value, followed by an error code. A typical API call will have the following syntax:
```go
value, error = db.function(parameters,...)
```

There are three fields that are usually present in a call:
* value - An object or value being retrieved by the function
* error - The error status
* db - An identifier or object that the function is being performance against

For instance, if the function is being executed against a database, then the identifier `db` is expected to be a handle, or pointer to the database object that would have been created earlier in the code.

In all examples, the `error` field will contain `nil` if the function succeeded, otherwise it will be a non-null value. All functions should be followed by logic to trap any errors that occur with the function call:
```go
db, error := sql.Open("go_ibm_db",*connStr);
if error != nil {
    return error;
}
```

<a name="OpenApi"></a>
## Open a Connection to a Database 

Syntax:
```go
db, err = Open(drivername,ConnectionString)
```
* `drivername` - Always set to `go_ibm_db`
* `connectionString` - The connection string for your database

For distributed platforms, the connection string is typically defined as: 
```
DATABASE=dbname;HOSTNAME=hostname;PORT=port;PROTOCOL=TCPIP;UID=username;PWD=passwd
```

Returns:

* `conn` - Connection handle used for subsequent calls
* `error` - Error details

Example:
```go
var connStr = flag.String("go_ibm_db", 
                          "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1")

db, err := sql.Open("go_ibm_db",*connStr);
```

<a name="CloseApi"></a>
## Close Database Connection

This function will close the specified database connection.

Syntax:
```go
db.Close()
```
* db - Current database handle

Example:
```go
defer db.Close()
```

<a name="QueryApi"></a> 
##  Issue a Query Against a Database

Syntax:
```
rows, err = db.Query("sql")
```

* db - Current database handle
* sql - The SQL that is going to be executed

Returns:
* rows - Answer set 
* error - Error details

```go
rows,error := db.Query("select * from ak")
```

<a name="PrepareApi"></a>
## Prepare a SQL Statement

Use this command to prepare a SQL statement for subsequent execution.

Syntax:
`stmt, error = db.Prepare(sqlquery)`

* `db` - The connection handle that was returned from the `open` connection call.
* `sqlquery` - SQL string that you want to prepare

Returns:

* `stmt` - Statement object used for subsequent calls
* `error` - Error details


```go
stmt, error := db.Prepare("select * from ak")
```

Once you prepare a statement, you could use the `Query` function to execute it.

<a name="ExecApi"></a> 
## Execute a DDL Statement
Use this statement to execute SQL that does not return rows. This is typically used for DDL statements rather than SQL.

Syntax:
```
db.Exec(sqlquery)
```
* `db` - The connection handle that was returned from the `open` connection call.
* `sqlquery` - SQL string that you want to execute

Returns:

* `error` - Error details

Note the use of the `_` operator to indicate any results are to be ignored from this call.

Example:
```go
_,error = db.Exec("create table ghh(a int, b float, c double,  d char, e varchar(30))")
```

<a name="BeginApi"></a> 
## Begin a SQL Transaction
When you begin a transaction, the program must issue either a `COMMIT` to complete the transaction, or a `ROLLBACK` to discard the changes.

Syntax:
```go
stmt, err = db.Begin()
```
* `db` - The connection handle that was returned from the `open` connection call.

Returns:

* `stmt` - Statement handle that has a commit scope
* `error` - Error details

Example:
```go
txid, err := db.Begin()
```

You would use the `stmt` handle when 

<a name="CommitApi"></a> 
## Commit a Transaction


Commit a current transaction.

Syntax:
```go
txid.Commit()
```

* `stmt` - The statement handle that was generated earlier by the `Begin()` function.

Returns:

* `txid` - Transaction handle
* `error` - Error details


func oper() error{
    fmt.Println("connecting to go-ibm_db");
    db, err := sql.Open("go-ibm_db",*connStr);
    if err != nil{
        return err;
    }
    

    defer db.Close()

    bg, err := db.Begin()
    if err != nil {
        return err
    }
    
    _,err=bg.Exec("create table ghh(a int,b float,c double,d char,e varchar(30))")
    if err != nil {
        return err
    }
    
    err = bg.Commit()
    if err != nil {
        return err
    }

    return nil
}
```

### <a name="RollbackApi"></a> 8) .Rollback()

Rollback a transaction.

```go


func oper() error{
    fmt.Println("connecting to go-ibm_db");
    db, err := sql.Open("go-ibm_db",*connStr);
    if err != nil {
        return err;
    }

    defer db.Close()
    bg, err := db.Begin()
    if err !=nil {
        return err
    }

    _,err=bg.Exec("create table ghh(a int,b float,c double,d char,e varchar(30))")
    if err != nil {
        return err
    }

    err = bg.Rollback()
    if err != nil {
        return err
    }
    
    return nil
}
```

### <a name="QueryRowApi"></a> 9) .QueryRow(sqlquery)

QueryRow executes a query that is expected to return at most one row.
If there are more rows then it will scan first and discards the rest.
 
```go

func oper()  error {
    id := 123
    var username string
    err := db.QueryRow("SELECT name FROM ak WHERE id=?", id).Scan(&username)
    if err != nil {
        return err
    }

    fmt.Printf("Username is %s\n", username)
    return nil
}

```

### <a name="ColumnsApi"></a> 10) .Columns()

Returns the column names.

Returns error if the rows are closed.

```go

func oper() error {
    fmt.Println("connecting to databse");
    db, err := sql.Open("go-ibm_db",*connStr);
    if err != nil {
        return err;
    }

    defer db.Close()

    st, err := db.Prepare("select * from ak")
    if err !=nil {
    return err
    }
    
    rows,err :=st.Query()
    if err != nil {
        return err
    }
    
    defer rows.Close()
    name11 := make([]string,1)
    name11, err = rows.Columns()
    fmt.Printf("%v",name11);
    return nil
}
```

### <a name="NextApi"></a> 11) .Next()

Prepares the next result row for reading with the scan api.

```go

func oper() error {
    fmt.Println("connecting to database");
    db, err:=sql.Open("go-ibm_db",*connStr);
    if err != nil {
        return err;
    }

    defer db.Close()
    rows,err := db.Query()
    if err != nil {
        return err
    }
    
    defer rows.Close()
    for rows.Next() {
        var t string
        var x string
        err = rows.Scan(&t, &x)
        if err != nil {
            return err
        }

        fmt.Printf("%v %v\n",t,x)
    }
    
    return nil
}
```

### <a name="ScanApi"></a> 12) .Scan(options)

copies the columns in the current row into the values pointed.

```go

func oper() error {
    fmt.Println("connecting to database");
    db, err := sql.Open("go-ibm_db", *connStr);
    if err != nil {
        return err;
    }

    defer db.Close()

    rows,err := db.Query()
    if err != nil {
        return err
    }

    defer rows.Close()
    for rows.Next() {
        var t string
        var x string
        err = rows.Scan(&t, &x)
        if err != nil {
            return err
        }

        fmt.Printf("%v %v\n",t,x)
    }
    return nil
}
```

## Summary
In summary, you've learned how to start, create, update and edit Jupyter notebooks. Jupyter notebooks are used extensively by the data science community, but it is finding its way into many other areas as well. If you are interested in what other applications use Jupyter notebooks, take a look at the list maintained on this web site: https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]